SETUP FEES COLLECTOR ON ALL PAIRS

1. GET ALL PAIRS
2. CHECK PAIR STATE TO BE ACTIVE
2. CHECK FEES COLLECTOR ADDRESS TO BE EMPTY IN PAIR
3. SET FEES COLLECTOR ADDRESS & 100% SPECIAL FEES
4. SET 0.1% SPECIAL FEES ON ALL ACTIVE PAIRS

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config

In [ ]:
from multiversx_sdk import Address
from context import Context
from contracts.contract_identities import PairContractVersion
from contracts.pair_contract import PairContract
from contracts.router_contract import RouterContract
from utils.contract_data_fetchers import PairContractDataFetcher, RouterContractDataFetcher

context = Context()

fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
router_contract_fetcher = RouterContractDataFetcher(Address.new_from_bech32(router_contract.address), context.network_provider.proxy.url)

pairs_addresses = router_contract_fetcher.get_data("getAllPairsManagedAddresses")

print(f"Pairs Count: {len(pairs_addresses)}")

counter = 0
for pair_address_raw in pairs_addresses[:25]:
    counter += 1
    print(f"Pair {counter}/{len(pairs_addresses)}")

    pair_address = Address.new_from_hex(pair_address_raw)

    pair_contract_fetcher = PairContractDataFetcher(pair_address, context.network_provider.proxy.url)

    pair_state = pair_contract_fetcher.get_data("getState")

    if pair_state != 1:
        continue

    fees_collector_address = pair_contract_fetcher.get_data("getFeesCollectorAddress")
    total_fee_percent = pair_contract_fetcher.get_data("getTotalFeePercent")
    special_fee_percent = pair_contract_fetcher.get_data("getSpecialFee")
    pair_contract = PairContract.load_contract_by_address(pair_address.to_bech32(), PairContractVersion.V2)

    print(f'{pair_address}: fees collector: {Address.new_from_hex(fees_collector_address).bech32()}; total_fee_percent: {total_fee_percent}; special_fee_percent: {special_fee_percent}')

    if Address.new_from_hex(fees_collector_address).is_empty() == True:
        pair_contract.add_fees_collector(context.deployer_account, context.network_provider.proxy, [fees_collector_contract.address, 100000])
    
    if int(special_fee_percent) != 100:
        pair_contract.set_fees_percents(context.deployer_account, context.network_provider.proxy, [int(total_fee_percent), 100])

Ownership takeover

In [ ]:
from contracts.staking_contract import StakingContract
staking_contracts = [
    "erd1qqqqqqqqqqqqqpgqjcatffrexdyd097ps5gvp4h8h9hsp2jkkp2sgewvwk",
    "erd1qqqqqqqqqqqqqpgq5x2x5xzm35x6nr4arrfl7zd47kz6xjzhkp2svp3z30",
    "erd1qqqqqqqqqqqqqpgqfnrln7ygyxdj96749e586lqwl8wr2fxskp2skn495u"
]
old_owner = "erd19uysnnzwnzm3klq8qzv7zprw2d3yj38hx0ac9cuqvl0j2jr5kp2sflvmyv"

for staking_address in staking_contracts:
    staking_contract = StakingContract.load_contract_by_address(staking_address)
    staking_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, old_owner)